<a href="https://colab.research.google.com/github/chrismckie/CSCI164/blob/main/ProblemSolvingSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import random
import heapq
import time

# Tile Sliding Domain: Initial State Space

In [15]:
# Initialize for 3x3 puzzles
StateDimension=3
InitialState = [1,2,3,4,5,6,0,7,8]
GoalState=[1,2,3,4,5,6,7,8,0]
Actions = lambda s: ['u', 'd', 'l', 'r']
Opposite=dict([('u','d'),('d','u'),('l','r'),('r','l'), (None, None)])

In [3]:
def Result(state, action):
  i = state.index(0)
  newState = list(state)
  row,col=i//StateDimension, i % StateDimension
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return newState
  if action=='u':
    l,r = row*StateDimension+col, (row-1)*StateDimension+col
  elif action=='d':
    l,r = row*StateDimension+col, (row+1)*StateDimension+col
  elif action=='l':
    l,r = row*StateDimension+col, row*StateDimension+col-1
  elif action=='r' :
    l,r = row*StateDimension+col, row*StateDimension+col+1
  newState[l], newState[r] = newState[r], newState[l]
  return newState

def PrintState(s):
  for i in range(0,len(s),StateDimension):
    print(s[i:i+StateDimension])

def LegalMove(state, action):
  i = state.index(0)
  row,col=i//StateDimension, i % StateDimension
  newState = state.copy()
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return False
  return True

In [4]:
def SingleTileManhattanDistance(tile, left, right):
  leftIndex = left.index(tile)
  rightIndex = right.index(tile)
  return (abs(leftIndex//StateDimension-rightIndex//StateDimension) +
          abs(leftIndex%StateDimension-rightIndex%StateDimension))

def ManhattanDistance(left, right):
  distances = [SingleTileManhattanDistance(tile, left, right)
     for tile in range(1, StateDimension**2)]
  ### print ("Distances= ", distances)
  return sum(distances)

In [5]:
def OutOfPlace(left, right):
  distances = [left[i]!=right[i] and right[i] != 0
     for i in range(StateDimension**2)]
  return sum(distances)

In [23]:
def Solution(node):
  if node.Parent==None:
    return []
  return Solution(node.Parent) + [node.Action]

# Random Walk

Take some random moves from a state and return the new state and the sequence of moves.

Do not include moves undoing last move, or having no effect.

In [6]:
def RandomWalk(state, steps):
  actionSequence = []
  actionLast = None
  for i in range(steps):
    action = None
    while action==None:
      action = random.choice(Actions(state))
      action = action if (LegalMove(state, action)
          and action!= Opposite[actionLast]) else None
    actionLast = action
    state = Result(state, action)
    actionSequence.append(action)
  return state, actionSequence

In [25]:
def ApplyMoves(actions, state):
  for action in actions:
    state = Result(state, action)
  return state

## Problem Class

INITIAL = InitialState  
IsGoal = Goal Test  
Actions = Actions List  
Result = Action Behavior  
ActionCost = Action Cost  

In [7]:
class Problem(object): pass

## Node

1.   List item
2.   List item


In [8]:
class Node(object):
  def __init__(self, state, parent=None, action=None, path_cost=0 ):
    self.State=state
    self.Parent=parent
    self.Action=action
    self.PathCost = path_cost

  def __str__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __repr__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __lt__(self, other):
    return self.PathCost < other.PathCost;

## Expand

In [9]:
def Expand(problem, node):
  ret = []
  s = node.State
  for action in problem.Actions(s):
    sPrime = problem.Result(s, action)
    cost =node.PathCost + problem.ActionCost(s,action,sPrime)
    ret.append(Node(sPrime, node, action, cost))
  return ret

## Breadth-First Search

In [10]:
def BreadthFirstSearch(problem):
  node = Node(tuple(problem.INITIAL))
  if problem.IsGoal(node.State):
    return node, 0
  Frontier = []
  Frontier.append(node)
  reached = set()
  reached.add(tuple(problem.INITIAL))
  nodesExpanded = 0
  while (Frontier):
    ### print([str(n) for n in Frontier])
    node = Frontier.pop(0)
    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      ### print (s, "IsGoal=", problem.IsGoal(s))
      if problem.IsGoal(s):
        return child, nodesExpanded
      if s not in reached:
        reached.add(s)
        Frontier.append(child)
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Best-First Search (For A* Search)

In [11]:
def BestFirstSearch(problem, f):
  node = Node(tuple(problem.INITIAL))
  Frontier = []
  heapq.heappush(Frontier,(f(node), node))
  reached = {}
  reached[tuple(problem.INITIAL)]=node
  nodesExpanded = 0
  while (Frontier):
    ##print([(x, str(n)) for (x,n) in Frontier])
    fValue, node = heapq.heappop(Frontier)
    ##print (node.State, "IsGoal=", problem.IsGoal(tuple(node.State)))
    if problem.IsGoal(tuple(node.State)):
      return node, nodesExpanded    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      if s not in reached or child.PathCost < reached[s].PathCost:
        reached[s] = child
        heapq.heappush(Frontier, (f(child), child))
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

# Test Problems

In [16]:
# Tile Sliding Definitions - Define IsGoal for 3x3 tiles
TileSliding = Problem()
TileSliding.INITIAL = InitialState
TileSliding.IsGoal = lambda s: s==(1,2,3,4,5,6,7,8,0)
TileSliding.Actions = Actions
TileSliding.Result=Result
TileSliding.ActionCost = lambda s, a, sPrime: 1

In [36]:
# Function that reworks problem list template (from original file) for testing multiple problems and different algorithms
def solveTestProblems(testProblems, numTiles, numSteps, algorithmName):
  Solutions = []
  print(numTiles, " Tiles: ", numSteps, " steps using ", algorithmName)
  for s in testProblems:
    TileSliding.INITIAL = s
    # Algorithm used depends on input
    if algorithmName == "Breadth-First Search":
      start_time = time.time()
      ret, cost = BreadthFirstSearch(TileSliding)
      end_time = time.time()
    elif algorithmName == "A* with Out-of-Place":
      AStarFb = lambda n: n.PathCost + OutOfPlace(n.State, GoalState)
      start_time = time.time()
      ret, cost = BestFirstSearch(TileSliding, AStarFb)
      end_time = time.time()
    elif algorithmName == "A* with Manhattan-Distance Heuristics":
      AStarFb = lambda n: n.PathCost + ManhattanDistance(n.State, GoalState)
      start_time = time.time()
      ret, cost = BestFirstSearch(TileSliding, AStarFb)
      end_time = time.time()
    sol = Solution(ret)
    elapsed_time = end_time - start_time
    print (sol)
    print ("-----------------------")
    print (TileSliding.INITIAL,'\n')
    print (ApplyMoves(sol, TileSliding.INITIAL))
    print ("Length of solution: ", len(sol))
    print ("Nodes Expanded=", cost)
    print("Time taken:", round(elapsed_time, 4), "seconds")
    print ("-----------------------")
    Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
  print ("-------")
  print (Solutions)
  print()


# 15 Generated Problems: 3x3 Tiles

In [30]:
# 3x3 Tile Problems
StateDimension = 3
GoalState=[1,2,3,4,5,6,7,8,0]

# Generalte Random Problems with RandomWalks from the Goal State
# Random Walk 5 Steps
test3x3_5stepsA = RandomWalk(GoalState, 5)
test3x3_5stepsB = RandomWalk(GoalState, 5)
test3x3_5stepsC = RandomWalk(GoalState, 5)

# Random Walk 10 Steps
test3x3_10stepsA = RandomWalk(GoalState, 10)
test3x3_10stepsB = RandomWalk(GoalState, 10)
test3x3_10stepsC = RandomWalk(GoalState, 10)

# Random Walk 20 Steps
test3x3_20stepsA = RandomWalk(GoalState, 20)
test3x3_20stepsB = RandomWalk(GoalState, 20)
test3x3_20stepsC = RandomWalk(GoalState, 20)

# Random Walk 40 Steps
test3x3_40stepsA = RandomWalk(GoalState, 40)
test3x3_40stepsB = RandomWalk(GoalState, 40)
test3x3_40stepsC = RandomWalk(GoalState, 40)

# Random Walk 80 Steps
test3x3_80stepsA = RandomWalk(GoalState, 80)
test3x3_80stepsB = RandomWalk(GoalState, 80)
test3x3_80stepsC = RandomWalk(GoalState, 80)

# Output for Testing
print("3x3 Random Problem: 5 steps")
PrintState(test3x3_5stepsA[0])
print("3x3 Random Problem: 10 steps")
PrintState(test3x3_10stepsA[0])
print("3x3 Random Problem: 20 steps")
PrintState(test3x3_20stepsA[0])
print("3x3 Random Problem: 40 steps")
PrintState(test3x3_40stepsA[0])

3x3 Random Problem: 5 steps
[1, 2, 3]
[7, 4, 5]
[8, 0, 6]
3x3 Random Problem: 10 steps
[7, 1, 3]
[5, 2, 6]
[0, 4, 8]
3x3 Random Problem: 20 steps
[1, 3, 8]
[4, 2, 5]
[0, 7, 6]
3x3 Random Problem: 40 steps
[0, 6, 2]
[8, 4, 3]
[1, 5, 7]


In [37]:
# Solve the randomly generated problems for 3x3 tiles
testProblem5Step = [
    test3x3_5stepsA[0],
    test3x3_5stepsB[0],
    test3x3_5stepsC[0]
]
testProblem10Step = [
    test3x3_10stepsA[0],
    test3x3_10stepsB[0],
    test3x3_10stepsC[0]
]
testProblem20Step = [
    test3x3_20stepsA[0],
    test3x3_20stepsB[0],
    test3x3_20stepsC[0]
]
testProblem40Step = [
    test3x3_40stepsA[0],
    test3x3_40stepsB[0],
    test3x3_40stepsC[0]
]
testProblem80Step = [
    test3x3_80stepsA[0],
    test3x3_80stepsB[0],
    test3x3_80stepsC[0]
]

solveTestProblems(testProblem5Step, "3x3", 5, "Breadth-First Search")
solveTestProblems(testProblem5Step, "3x3", 10, "Breadth-First Search")
solveTestProblems(testProblem5Step, "3x3", 20, "Breadth-First Search")
solveTestProblems(testProblem5Step, "3x3", 40, "Breadth-First Search")
solveTestProblems(testProblem5Step, "3x3", 80, "Breadth-First Search")

solveTestProblems(testProblem5Step, "3x3", 5, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "3x3", 10, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "3x3", 20, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "3x3", 40, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "3x3", 80, "A* with Out-of-Place")

solveTestProblems(testProblem5Step, "3x3", 5, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "3x3", 10, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "3x3", 20, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "3x3", 40, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "3x3", 80, "A* with Manhattan-Distance Heuristics")

3x3  Tiles:  5  steps using  Breadth-First Search
['l', 'u', 'r', 'r', 'd']
-----------------------
[1, 2, 3, 7, 4, 5, 8, 0, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  5
Nodes Expanded= 28
Time taken: 0.0002 seconds
-----------------------
['l', 'd', 'd', 'r', 'r']
-----------------------
[2, 0, 3, 1, 5, 6, 4, 7, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  5
Nodes Expanded= 25
Time taken: 0.0003 seconds
-----------------------
['l', 'd', 'r', 'd', 'r']
-----------------------
[2, 0, 3, 1, 4, 6, 7, 5, 8] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  5
Nodes Expanded= 27
Time taken: 0.0002 seconds
-----------------------
-------
[('123745806', 'lurrd', 28), ('203156478', 'lddrr', 25), ('203146758', 'ldrdr', 27)]

3x3  Tiles:  10  steps using  Breadth-First Search
['l', 'u', 'r', 'r', 'd']
-----------------------
[1, 2, 3, 7, 4, 5, 8, 0, 6] 

[1, 2, 3, 4, 5, 6, 7, 8, 0]
Length of solution:  5
Nodes Expanded= 28
Time taken: 0.0002 seconds
-----------------------
[

# 15 Generated Problems: 4x4 Tiles

In [38]:
# Tile Sliding Definitions - Redefine IsGoal for for 4x4 tiles
TileSliding = Problem()
TileSliding.INITIAL = InitialState
TileSliding.IsGoal = lambda s: s==(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
TileSliding.Actions = Actions
TileSliding.Result=Result
TileSliding.ActionCost = lambda s, a, sPrime: 1

In [40]:
# 4x4 Tile Problems
StateDimension = 4
GoalState=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]

# Generalte Random Problems with RandomWalks from the Goal State
# Random Walk 5 Steps
test4x4_5stepsA = RandomWalk(GoalState, 5)
test4x4_5stepsB = RandomWalk(GoalState, 5)
test4x4_5stepsC = RandomWalk(GoalState, 5)

# Random Walk 10 Steps
test4x4_10stepsA = RandomWalk(GoalState, 10)
test4x4_10stepsB = RandomWalk(GoalState, 10)
test4x4_10stepsC = RandomWalk(GoalState, 10)

# Random Walk 20 Steps
test4x4_20stepsA = RandomWalk(GoalState, 20)
test4x4_20stepsB = RandomWalk(GoalState, 20)
test4x4_20stepsC = RandomWalk(GoalState, 20)

# Random Walk 40 Steps
test4x4_40stepsA = RandomWalk(GoalState, 40)
test4x4_40stepsB = RandomWalk(GoalState, 40)
test4x4_40stepsC = RandomWalk(GoalState, 40)

# Random Walk 80 Steps
test4x4_80stepsA = RandomWalk(GoalState, 80)
test4x4_80stepsB = RandomWalk(GoalState, 80)
test4x4_80stepsC = RandomWalk(GoalState, 80)

# Output for Testing
print("4x4 Random Problem: 5 steps")
PrintState(test4x4_5stepsA[0])
print("4x4 Random Problem: 10 steps")
PrintState(test4x4_10stepsA[0])
print("4x4 Random Problem: 20 steps")
PrintState(test4x4_20stepsA[0])
print("4x4 Random Problem: 40 steps")
PrintState(test4x4_40stepsA[0])

4x4 Random Problem: 5 steps
[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 12, 15]
[13, 14, 0, 11]
4x4 Random Problem: 10 steps
[1, 2, 3, 4]
[5, 6, 7, 8]
[10, 13, 15, 11]
[9, 14, 12, 0]
4x4 Random Problem: 20 steps
[1, 6, 7, 2]
[5, 10, 3, 4]
[0, 9, 11, 8]
[13, 14, 15, 12]
4x4 Random Problem: 40 steps
[3, 4, 0, 6]
[2, 1, 12, 15]
[10, 11, 5, 7]
[14, 9, 13, 8]


In [41]:
# Solve the randomly generated problems for 3x3 tiles
testProblem5Step = [
    test4x4_5stepsA[0],
    test4x4_5stepsB[0],
    test4x4_5stepsC[0]
]
testProblem10Step = [
    test4x4_10stepsA[0],
    test4x4_10stepsB[0],
    test4x4_10stepsC[0]
]
testProblem20Step = [
    test4x4_20stepsA[0],
    test4x4_20stepsB[0],
    test4x4_20stepsC[0]
]
testProblem40Step = [
    test4x4_40stepsA[0],
    test4x4_40stepsB[0],
    test4x4_40stepsC[0]
]
testProblem80Step = [
    test4x4_80stepsA[0],
    test4x4_80stepsB[0],
    test4x4_80stepsC[0]
]

solveTestProblems(testProblem5Step, "4x4", 5, "Breadth-First Search")
solveTestProblems(testProblem5Step, "4x4", 10, "Breadth-First Search")
solveTestProblems(testProblem5Step, "4x4", 20, "Breadth-First Search")
solveTestProblems(testProblem5Step, "4x4", 40, "Breadth-First Search")
solveTestProblems(testProblem5Step, "4x4", 80, "Breadth-First Search")

solveTestProblems(testProblem5Step, "4x4", 5, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "4x4", 10, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "4x4", 20, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "4x4", 40, "A* with Out-of-Place")
solveTestProblems(testProblem5Step, "4x4", 80, "A* with Out-of-Place")

solveTestProblems(testProblem5Step, "4x4", 5, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "4x4", 10, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "4x4", 20, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "4x4", 40, "A* with Manhattan-Distance Heuristics")
solveTestProblems(testProblem5Step, "4x4", 80, "A* with Manhattan-Distance Heuristics")

4x4  Tiles:  5  steps using  Breadth-First Search
['r', 'u', 'l', 'd', 'r']
-----------------------
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 13, 14, 0, 11] 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
Length of solution:  5
Nodes Expanded= 54
Time taken: 0.0005 seconds
-----------------------
['r', 'd', 'l', 'd', 'r']
-----------------------
[1, 2, 3, 4, 5, 6, 0, 7, 9, 10, 12, 8, 13, 14, 11, 15] 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
Length of solution:  5
Nodes Expanded= 71
Time taken: 0.0005 seconds
-----------------------
['d', 'l', 'd', 'd', 'r']
-----------------------
[1, 2, 3, 0, 5, 6, 8, 4, 9, 10, 7, 12, 13, 14, 11, 15] 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
Length of solution:  5
Nodes Expanded= 23
Time taken: 0.0001 seconds
-----------------------
-------
[('1234567891012151314011', 'ruldr', 54), ('1234560791012813141115', 'rdldr', 71), ('1230568491071213141115', 'dlddr', 23)]

4x4  Tiles:  10  steps using  Breadth-First Search
[

# Analysis:

Based on the results of both the 3x3 tile puzzle and the 4x4 tile puzzle, there is a significant difference in the performance of the algorithms.

Regardless of the size of the puzzle, the BFS performed poorly compared to the others. Both the number of steps increasing and the number of tiles increasing led to an increase in time computing the solution, as well as the number of nodes expanded. BFS can solve the problems optimally, but will exponentially get worse as the problems become more complex due to finite memory and time.

For the A* algorithms, performance was much better, and both did better than BFS. A* with out-of-place consistently performed quickly and found the optimal solution. In addition, the number of nodes expanded averaged between 5 and 8 regardless of steps, siginficantly better than BFS, which would get near 100 at times.

A* with Manhattan-Distance performed slighty better than A* with out-of-place, making it the best algorithm for this task. Computation was fast and the solutions were optimal. In addition, the number of expanded nodes stayed near 5 regardless of steps or tile size, making it the best performing algorithm.

Overall, this test indicate that search is a valuable tool for Artificial Intelligence, but the search algorithm becomes more important as the problem increases in complexity. BFS can do well for simple tasks, but more complicated problems will need A* with heuristics, or a similar algorithm, in order to keep the problem efficient.